In [1]:
# Initialization. Auto update changed modules so the kernel does not need to be restarted after changes
%load_ext autoreload
%autoreload 2

# Set device udid. If your device udid is different, run adb devices and change the udid
device_udid = "emulator-5554"


# Exercise 2: Add new functionality
In this exercise, you will add new functionality to the camera app.
Currently, the camera class can only take a picture. In this exercise, we will learn how to switch the camera
and take a video.

Try to execute below code, what happens?


In [6]:
import os, time
os.chdir('../..')
from puma.apps.android.open_camera.open_camera import OpenCameraActions

camera = OpenCameraActions(device_udid)

camera.take_picture()
time.sleep(2)
camera.switch_camera()
time.sleep(2)
camera.take_picture()

`switch_camera` has not been implemented yet. That is what you will do in this exercise. When adding new
functionality to Puma, you will follow the following steps:
- Manually explore the actions you want to automate on the device to get an idea of the steps you need to
  implement
- Use Appium Inspector to identify the UI elements you want to interact with
- Write the Appium code to select the element and interact with it using XPATH

We will take you through these steps below.


# How Appium interacts with the App UI

Before we get started we need to udnerstand how Appium interacts with the UI. On Android the UI consists
of elements in a hierarchy, much like an XML document. Appium fetches this hierarchy through the Android
Accesability API and exposes this as an XML document to the user. Usually, only visible elements are
included, as invisible elements should not be loaded into the Android Accesibility service. However, not
all apps are equally well written, and some apps laod in elements too early, while other apps do not
load relevant data into the Accesability Service, leaving us navigating XML elements without any contents
or attributes.

Lucky for us, most of the time there's a few attributes we can use to interact with the desired UI element.
Let's take a look at the following UI:

<img src="resources/whatsapp.png" alt="Alt Text" width="300">

If we retrieve the XML element representing the send button (how we do that will be explained later in this
exercise), it looks like this:
```XML
<android.widget.FrameLayout resource-id="com.whatsapp:id/send_container">
    <android.widget.ImageButton content-desc="Send" resource-id="com.whatsapp:id/send"/>
<android.widget.FrameLayout/>
```
As you can see, the button itself is an `ImageButton` element nested inside a `FrameLayout` element.
The good news is that either can be clicked to interact with the button, so we don't need to worry about which
element we want.
You could idenitify the elements based on the element class, but in this case this will only work when there's
just one `ImageButton`. As UIs can become very complex, we usually want something more specific to go by, which
is why we usually also look at attributes.
Either of these elements can be identified by their `resource-id`, and the inner element also has a
`content-desc` which we can use to identify it. Of these two, the preference usually goes to `resource-id`, as
this value is indendent of the configured system language.

If we take a look at one of the sent messages, we see the following XML, containing a third attribute that can
be useful:
```XML
<android.widget.FrameLayout resource-id="com.whatsapp:id/conversation_text_row">
    <android.widget.TextView text="Hello there Alice, long time no see! How have you been??" resource-id="com.whatsapp:id/message_text"/>
        <android.widget.LinearLayout resource-id="com.whatsapp:id/date_wrapper">
            <android.widget.TextView text="1:21 PM" resource-id="com.whatsapp:id/date"/>
            <android.widget.ImageView content-desc="Delivered" resource-id="com.whatsapp:id/status"/>
        </android.widget.LinearLayout>
    </android.widget.FrameLayout>
</android.widget.LinearLayout>
```
Here we can read the message contents in the `text` attribute, as well as the time the message was sent. Keep 
in mind that the `text` attribute is even more volatile than the `content-desc`, as it might be determined by
both the system language and user input.

So, to summarize:
* The Android UI is exposed as an XML document through Appium
* Selecting elements can be done by looking at the element class and the attribute values
* In practice, the attributes `resource-id`, `content-desc` and `text` prove to be the most useful

# Appium Python Client
To use Appium in Python, Puma uses the Appium Python Client. The Appium Python Client works by sending commands to the Appium Server, which then communicates with the emulator. The interaction follows the WebDriver protocol, allowing automation of mobile applications similarly to how Selenium automates web browsers. First a webdriver session is initialized (see the AndroidAppiumActions class for the implementation). The webdriver has some functions to interact with elements, as well as retrieving information:
```python
# Open an app
driver.activate_app(app_package)

# Press the Android home key
driver.press_keycode(AndroidKey.HOME)

# Find an element in the current view
driver.find_element(by=AppiumBy.XPATH, value=xpath)

# Swipe from one coordinate to another
driver.swipe(start_x, start_y, end_x, end_y)
```

Commands can be executed on the elements, such as clicking and sending keys (_i.e._ typing text in a text box.):
```python
shutter_element = self.driver.find_element(by=AppiumBy.XPATH, value=xpath)
shutter_element.click()

text_box = self.driver.find_element(by=AppiumBy.XPATH, value=xpath)
text_box.send_keys("This is the text to insert into the text box")
```
You can see we use a certain string called `xpath` to select the elements. We'll get to its contents below.

# Finding XML elements
As seen in the previous example, finding elements can be done with `webdriver`.find_element(). The element can be found in different ways:

```python
driver.find_element(by=AppiumBy.ID, value="id")
driver.find_element(by=AppiumBy.CLASS_NAME, value="class_name")
driver.find_element(by=AppiumBy.ACCESSIBILITY_ID, value="accessibility_id")
driver.find_element(by=AppiumBy.XPATH, value="xpath")
```

As you can see, there are several ways to find elements. The first 3 are based on selecting a specific attribute. The last one in based on XPath. In Puma, we prefer to use XPath, as it is the most flexible and enables us to make more complex queries to select elements. Moreover, when something changes in a new version of the application, only the value needs to be updated.


# Appium inspector
Appium Inspector is a GUI assistant tool for Appium, providing visual inspection of the application.
1. Open Appium inspector.
2. Under `Saved Capability Sets`, select `Emulator` and press `Start Session`. If you do not see saved capabilities, please refer to `workshop/exercises/resources/appium_inspector_capability_set.json` and add it.
You now see your emulator screen on the left pane, here you can select elements. In the middle pane you see the xml representation of the current view in the app. On the right pane you see the information about the selected element.
3. On the emulator, open the Open Camera app (not the fregular camera!). Make sure you're in the app home screen (the section where you can take a picture). Refresh AppiumInspector to update the screen if necessary.
4. In Appium inspector, select the settings button (the cogwheel) in the top-right corner and look at the information about the element in the right pane.

<details>
  <summary style="font-size: 24px;">What is the class of this element? (click to view answer)</summary>
  This element is of the class <tt>android.widget.ImageButton</tt>. As we can see in the UI, there's other buttons of the same class, so
    selecting the button based on its class alone will not be enough.
</details>

<details>
  <summary style="font-size: 24px;">What attribute has the most uniquely identifying value? (click to view answer)</summary>
  Since the class is not enough to select this element reliably, we need to also look at its attributes, and there's two of them:
    <ul>
        <li><tt>content-desc</tt>, with value <tt>Settings</tt></li>
        <li><tt>resource-id</tt>, with value <tt>net.sourceforge.opencamera:id/settings</tt></li>
    </ul>
  Both seem like great candidates to select the UI element. But what do you think will happen to the <tt>content-desc</tt> when the System language is set to, say, Spanish?
<br>
You can do this if you want and see what happens, but if you want to save time we'll just tell you: the value of <tt>content-desc</tt> will change to <tt>Ajustes</tt>. So the most reliable and unique attribute of the two is <tt>resource-id</tt>. By using this, you will ensure your Puma ccode works regardless of the System Language.
<br>
In some apps however, there will not be a <tt>resource-id</tt>, and only a <tt>content-desc</tt> or <tt>text</tt> attribute is available, both being language-dependent. In htose case we do not have a choice, and your code will only work in the supproted language. For this reason, the Puma documentation recommends setting the UI language to English.
</details>

# XPath expressions
If we know the class and attribute of the settings button, we are now able to click it. all we need to do is create
an XPath expression to select the button. [XPath](https://www.w3schools.com/xml/xpath_intro.asp) is a language allowing
you to navigate XML documents. In Puma and Appium you can use it to search for the correct element(s). Below is a basic
introduction, but for a more detailed overview, see [W3Schools.com](http://www.w3schools.com/Xml/xpath_intro.asp).

## Selecting based on class and attribute

When looking for an element of a given class `foo`, with an attribute `bar` which has value `baz`, you can use the
following XPATH expression:
```
//foo[@bar='baz']
```
* the doulbe slash `//` means 'an element anywhere under the root of the document'
* then `foo` describes the desired element class
* the brackets `[]` contain a predicate that applies a filter to the selected elements
* The predicate `@bar='baz'` means we only want elemnts with an attribute `bar` which has value `baz`

**Keep in mind that every matching element will be matched by the XPath expression.** So if your 

## Hierarchical selection

Sometimes it's difficult to select the exact eleemnt you want absed on its class and attributes alone. In such
cases you can also describe the hierarchy under which the element exists:
```
//foo/bar//baz
```
This expression measn *select a `baz` element which occurs anywhere under a bar element which should exist directly
under a `foo` element which exists anywhere in the document*.
The key takeaway here is that a single slash `/` means that the element should be a direct descendant, while a
double slash `//` means it should be a descendant on any level.

This can be combined with attributes wherever you want:
```
//foo[@fizz='buzz']/bar//baz[@qwerty='azerty']
```

Also, you can select the n-th decendant of a certain type:
```
//foo//bar[2]
```
This will take the **second** `bar` element directly under `foo`. **XPath counting starts at 1.**
Keep in mind that this will return **every** `bar` element that is the second `bar` element under a `foo` element.
So if there's multiple `foo` elements that have 2 or more `bar` descendants, this XPath expression matches multiple
elements.

## XPath operators and functions

Latsly, we can use more complex predicates by using [XPATH Operators](http://www.w3schools.com/Xml/xpath_operators.asp)
and [XPath functions](https://developer.mozilla.org/en-US/docs/Web/XML/XPath/Reference/Functions).
These allow us to filter elements in a more powerfull way. There's many operators and functions, but these are the
most essential for use in Puma:
* `and` operator to combine multiple predicates: `//foo[@bar='1' and @buz='2']`
* `contains()` for broader predicates: `//foo[contains(@bar, 'buz')]` (`bar` attribute should contains the string `buz`)
* `lower-case()` for case insensitive matching: `//foo[lower-case(@bar) = 'baz']` (`bar` attribute should cointain `baz`, but ignore casing)

These can all be contained to make powerful XPath expressions, for example:

`//contact[contains(lower-case(@name), 'jack') and @status='online']`


# Back to our camera! Implementing the switch camera button

Advanced: zoom in/out by pinching